## Outils pour la manipulation d'images et librairies.


In [12]:
import PIL
from PIL import Image
import numpy as np
import os
from math import log10, sqrt

def load(filename):
    toLoad= Image.open(filename)
    return np.asarray(toLoad)


def psnr(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

def dct2(a):
    return sp.fft.dct( sp.fft.dct( a, axis=0, norm='ortho' ), axis=1, norm='ortho' )

def idct2(a):
    return sp.fft.idct( sp.fft.idct( a, axis=0 , norm='ortho'), axis=1 , norm='ortho')



## Normalisation de l'image (YCbCr et padding)

In [13]:
def Y(mat):
    MatY = np.empty((mat.shape[0], mat.shape[1], 1))
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            MatY[i, j] = 0.299 * mat[i, j, 0] + 0.587 * mat[i, j, 1] + 0.114 * mat[i, j, 2]
    return MatY

def Cb(mat):
    MatCb = np.empty((mat.shape[0], mat.shape[1], 1))
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            MatCb[i, j] = -0.1687 * mat[i, j, 0] - 0.3313 * mat[i, j, 1] + 0.5 * mat[i, j, 2] + 128
    return MatCb

def Cr(mat):
    MatCr = np.empty((mat.shape[0], mat.shape[1], 1))
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            MatCr[i, j] = 0.5 * mat[i, j, 0] - 0.4187 * mat[i, j, 1] - 0.0813 * mat[i, j, 2] + 128
    return MatCr

def YCbCr(mat):
    MatYCbCr = np.empty(mat.shape)
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            R = 0.299 * mat[i, j, 0] + 0.587 * mat[i, j, 1] + 0.114 * mat[i, j, 2]
            Cb = -0.1687 * mat[i, j, 0] - 0.3313 * mat[i, j, 1] + 0.5 * mat[i, j, 2] + 128
            Cr = 0.5 * mat[i, j, 0] - 0.4187 * mat[i, j, 1] - 0.0813 * mat[i, j, 2] + 128
            MatYCbCr[i, j] = [R, Cb, Cr]
    return MatYCbCr

def RGB2(mat):
    MatRGB = np.empty((mat.shape[0], mat.shape[1], 3))
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            R = mat[i, j, 0] + 1.402 * (mat[i, j, 2] - 128)
            G = mat[i, j, 0] - 0.34414 * (mat[i, j, 1] - 128) - 0.71414 * (mat[i, j, 2] - 128)
            B = mat[i, j, 0] + 1.772 * (mat[i, j, 1] - 128)
            MatRGB[i, j] = (np.uint8(np.clip(R, 0.0, 255.0)), np.uint8(np.clip(G, 0.0, 255.0)), np.uint8(np.clip(B, 0.0, 255.0)))
    return MatRGB

def add_padding(image, pad_size):
    if isinstance(pad_size, int):
        pad_size = (pad_size, pad_size)  # Si un seul entier est donné, le même padding sera ajouté dans les deux dimensions

    padded_image = np.pad(image, ((pad_size[0], pad_size[0]), (pad_size[1], pad_size[1]), (0, 0)), mode='constant')

    return padded_image


def remove_padding(padded_image, pad_size):
    if isinstance(pad_size, int):
        pad_size = (pad_size, pad_size)  # Si un seul entier est donné, le même padding sera éliminé dans les deux dimensions

    image = padded_image[pad_size[0]:-pad_size[0], pad_size[1]:-pad_size[1], :]

    return image

## Découpage en blocs et compression

In [14]:
# Fonction Matrice sous-echantillonnee 
#question 4
def matrice_sousechantillon(mat):
    matrice_se = mat[::2, ::2]
    return matrice_se

#Question 5
def matrice_2d(matrice):
    matrice_double = np.repeat(matrice, 2, axis=1)
    return matrice_double


## Écriture dans un fichier

## Décompression

## Tests 

In [15]:
test = load("test.png")
Image.fromarray(test,'RGB').show()
# Charger l'image
image = load("test.png")

# Ajouter le padding
pad_size = 10
padded_image = add_padding(image, pad_size)

subsampled_yCbCr = matrice_sousechantillon(YCbCr(padded_image))
padded_yCbCr = YCbCr(padded_image)
restored_yCbCr = remove_padding(padded_yCbCr, pad_size)

subsampled_RGB = matrice_sousechantillon(RGB2(padded_image))
padded_RGB = RGB2(padded_image)
restored_RGB = remove_padding(padded_RGB, pad_size)


test = load("test.png")
# Afficher l'image convertie en YCbCr
Image.fromarray(restored_yCbCr.astype(np.uint8), 'YCbCr').show()

# Afficher l'image convertie en RGB
Image.fromarray(restored_RGB.astype(np.uint8), 'RGB').show()

# Charger l'image
image = Image.open("test.png")
image_array = np.array(image)

# Sous-échantillonnage
subsampled_image = image_array[::2, ::2]

# Multiplier par deux la deuxième dimension
doubled_image = matrice_2d(subsampled_image)

# Afficher les images
Image.fromarray(image_array).show()
Image.fromarray(subsampled_image).show()
Image.fromarray(doubled_image).show()